In [1]:
import pandas as pd
import arcpy
from arcgis import GIS
gis = GIS()
import json

In [2]:
fnSegments = 'data/segments/Segments_WF - 2023-08-01.shp'

seg_cols = ['SEGID','CO_NAME','PLANAREA','AADT2019','SHAPE']

# USE ALL PLANAREAS since this app will be for entire state
# filter by PLANAREA in segments shapefile
#filterPlanArea = ['WFRC','MAG'] # must be an array... if only single item, the still include []

In [3]:
# Prep flag categories

# flag name must be short enough to be in dbf column name
dfFlags = pd.DataFrame([
    ['FL_NOREV' , 'Not reviewed'                          ,"([ADJ2023]==-1) | ([ADJ2028]==-1) | ([ADJ2032]==-1) | ([ADJ2042]==-1) | ([ADJ2050]==-1)"],
    ['FL_LTPRV' , 'Less than previous forecast year'      ,"(([MF2023]+[ADJ2023])<[AADT2019]) | (([MF2028]+[ADJ2028])<([MF2023]+[ADJ2023])) | (([MF2032]+[ADJ2032])<([MF2028]+[ADJ2028])) | (([MF2042]+[ADJ2042])<([MF2032]+[ADJ2032])) | (([MF2050]+[ADJ2050])<([MF2042]+[ADJ2042]))"],
    ['FL_502X19', '2050 more than twice 2019'             ,"(([MF2050]+[ADJ2050])>(2*[AADT2019]+[ADJ2050]))"],
    ['FL_ZERO'  , 'Zero volume'                           ,"(([MF2023]+[ADJ2023])==0) | (([MF2028]+[ADJ2028])==0) | (([MF2032]+[ADJ2032])==0) | (([MF2042]+[ADJ2042])==0) | (([MF2050]+[ADJ2050])==0)"]
], columns=('flagName','flagDescription','flagCriteria'))

dfFlags.to_json('_site/data/flags.json', orient='records')

In [4]:
# read in segments, filter, and select only key columns
sdfSegments = pd.DataFrame.spatial.from_featureclass(fnSegments)
#sdfSegments = sdfSegments[sdfSegments['PLANAREA'].isin(filterPlanArea)]
sdfSegments = sdfSegments[seg_cols]

display(sdfSegments)

,SEGID,CO_NAME,PLANAREA,AADT2019,SHAPE
0,0006_000.0,MILLARD,UDOT,415.0,"{""paths"": [[[236177.7000000002, 4327541.25], [..."
1,0006_000.7,MILLARD,UDOT,415.0,"{""paths"": [[[237241.1799999997, 4327399.720000..."
2,0006_016.0,MILLARD,UDOT,415.0,"{""paths"": [[[261403.90000000037, 4327045.4], [..."
3,0006_046.0,MILLARD,UDOT,372.0,"{""paths"": [[[305380.16000000015, 4325741.85], ..."
4,0006_060.2,MILLARD,UDOT,372.0,"{""paths"": [[[324618, 4337936.9], [324647.40000..."
...,...,...,...,...,...
8722,UTA_7132,,WFRC,0.0,"{""paths"": [[[424262.6891235444, 4508606.236747..."
8723,UTA_7316,,WFRC,0.0,"{""paths"": [[[420773.38413784996, 4495314.95905..."
8724,UTA_7320,,WFRC,0.0,"{""paths"": [[[419033.24434054515, 4494684.05276..."
8725,UTA_7328,,WFRC,0.0,"{""paths"": [[[415002.73054989125, 4492771.26989..."


In [5]:
# export segment JSON

sdfSegments.loc[sdfSegments['SEGID']=='2322_004.5', 'PLANAREA'] = 'WFRC'

sdfSegments[[x for x in seg_cols if x != 'SHAPE']].to_json('_site/data/segments.json', orient='records')

In [6]:
# read in intermediate data
dfModVolAdj = pd.read_csv('intermediate/model-forecasts.csv')
dfModVolAdj['YEAR'] = dfModVolAdj['YEAR'].astype(str)
dfModVolAdj['YEAR'] = 'M' + dfModVolAdj['YEAR']

dfModVolAdj_pvModAadt = dfModVolAdj.pivot_table(index=['SEGID','FAC_WDAVG','aadtAdjFactor'], columns='YEAR', values='modAADT')
dfModVolAdj_pvModAadt.reset_index(inplace=True)

dfModVolAdj = pd.read_csv('intermediate/model-forecasts.csv')
dfModVolAdj['YEAR'] = dfModVolAdj['YEAR'].astype(str)
dfModVolAdj['YEAR'] = 'MF' + dfModVolAdj['YEAR']

dfModVolAdj_pvModAadtAdj = dfModVolAdj.pivot_table(index=['SEGID','FAC_WDAVG','aadtAdjFactor'], columns='YEAR', values='modForecast')
dfModVolAdj_pvModAadtAdj.reset_index(inplace=True)

dfModVolAdj = pd.DataFrame.merge(dfModVolAdj_pvModAadt, dfModVolAdj_pvModAadtAdj, on=('SEGID','FAC_WDAVG','aadtAdjFactor'))


dfModVolAdj

YEAR,SEGID,FAC_WDAVG,aadtAdjFactor,M2023,M2028,M2032,M2042,M2050,MF2023,MF2028,MF2032,MF2042,MF2050
0,0006_146.9,0.9840,-18.0,1671.0,1738.0,1892.0,2120.0,2409.0,1700.0,1700.0,1900.0,2100.0,2400.0
1,0006_149.9,0.9840,856.0,1799.0,2012.0,2075.0,2652.0,5357.0,2700.0,2900.0,2900.0,3500.0,6200.0
2,0006_150.6,0.9840,657.0,1989.0,2209.0,2281.0,2891.0,5598.0,2600.0,2900.0,2900.0,3500.0,6300.0
3,0006_152.6,0.9840,-119.0,2719.0,2988.0,3080.0,3756.0,6225.0,2600.0,2900.0,3000.0,3600.0,6100.0
4,0006_152.9,0.9593,-124.0,4230.0,4700.0,4991.0,5675.0,8072.0,4100.0,4600.0,4900.0,5600.0,7900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3717,WFRC_8259,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3718,WFRC_8260,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3719,WFRC_8261,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3720,WFRC_8263,1.0924,2427.0,5503.0,6511.0,6667.0,6888.0,7125.0,7900.0,8900.0,9100.0,9300.0,9600.0


In [7]:
# join segment data and forecast data
_sdf = pd.DataFrame.merge(dfModVolAdj, sdfSegments, on='SEGID', how='right')

# add columns for Adjustments - SET AS -1 to initialize... CODE RECOGNIZES #1 
_sdf['ADJ2023'] = -1
_sdf['ADJ2028'] = -1
_sdf['ADJ2032'] = -1
_sdf['ADJ2042'] = -1
_sdf['ADJ2050'] = -1

# add flag columns
# Loop through the rows of 'dfFlags'
for index, row in dfFlags.iterrows():
    # Get the flag name from the 'flagName' column
    flag_name = row['flagName']

    # Get the flag criteria from the 'flagCriteria' column (if needed)
    flag_criteria = row['flagCriteria']

    # Add a new column to 'dfSegs' with the flag name and set it equal to evaluated criteria, prepare criteria string with replace
    expression = "_sdf['" + flag_name + "'] = " + flag_criteria.replace("[", "_sdf['").replace("]", "']")
    print(expression)

    # execute expression!
    exec(expression)
    #print(result)  # Output will be 11
    

_sdf.fillna(0,inplace=True)
_sdf['NOTES'] = ""

sdfSegmentsWData = _sdf

# export
sdfSegmentsWData.spatial.to_featureclass('results/ForecastSegments/ForecastSegments.shp',sanitize_columns=False)

_sdf['FL_NOREV'] = (_sdf['ADJ2023']==-1) | (_sdf['ADJ2028']==-1) | (_sdf['ADJ2032']==-1) | (_sdf['ADJ2042']==-1) | (_sdf['ADJ2050']==-1)
_sdf['FL_LTPRV'] = ((_sdf['MF2023']+_sdf['ADJ2023'])<_sdf['AADT2019']) | ((_sdf['MF2028']+_sdf['ADJ2028'])<(_sdf['MF2023']+_sdf['ADJ2023'])) | ((_sdf['MF2032']+_sdf['ADJ2032'])<(_sdf['MF2028']+_sdf['ADJ2028'])) | ((_sdf['MF2042']+_sdf['ADJ2042'])<(_sdf['MF2032']+_sdf['ADJ2032'])) | ((_sdf['MF2050']+_sdf['ADJ2050'])<(_sdf['MF2042']+_sdf['ADJ2042']))
_sdf['FL_502X19'] = ((_sdf['MF2050']+_sdf['ADJ2050'])>(2*_sdf['AADT2019']+_sdf['ADJ2050']))
_sdf['FL_ZERO'] = ((_sdf['MF2023']+_sdf['ADJ2023'])==0) | ((_sdf['MF2028']+_sdf['ADJ2028'])==0) | ((_sdf['MF2032']+_sdf['ADJ2032'])==0) | ((_sdf['MF2042']+_sdf['ADJ2042'])==0) | ((_sdf['MF2050']+_sdf['ADJ2050'])==0)


'e:\\GitHub\\Traffic-Volume-Forecasts\\results\\ForecastSegments\\ForecastSegments.shp'

In [8]:
sdfSegmentsWData

,SEGID,FAC_WDAVG,aadtAdjFactor,M2023,M2028,M2032,M2042,M2050,MF2023,MF2028,...,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,FL_NOREV,FL_LTPRV,FL_502X19,FL_ZERO,NOTES
0,0006_000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,True,False,False,False,
1,0006_000.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,True,False,False,False,
2,0006_016.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,True,False,False,False,
3,0006_046.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,True,False,False,False,
4,0006_060.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,True,False,False,False,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8722,UTA_7132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,True,False,False,False,
8723,UTA_7316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,True,False,False,False,
8724,UTA_7320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,True,False,False,False,
8725,UTA_7328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1,-1,-1,-1,-1,True,False,False,False,


In [9]:
# convert csvs to jsons
lstSegidFiles = ['aadt', 'linear-forecasts', 'model-forecasts', 'previous-forecasts']
lstNonSegidFiles = ['aadt-sources', 'previous-forecasts-sources', 'projection-groups']

for file in lstSegidFiles + lstNonSegidFiles:
    # Read CSV file
    pd.read_csv('intermediate/' + file + '.csv').to_json('_site/data/' + file + '.json', orient='records')